# Multiclass Classification

## News Group Classification

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset = 'all', random_state = 2021)

### Searching Data

In [3]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
news.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
pd.Series(news.target).value_counts().sort_index()

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

In [7]:
len(news.data)

18846

In [8]:
print(news.data[0])

From: dagibbs@quantum.qnx.com (David Gibbs)
Subject: Re: Countersteering sans Hands
Organization: QNX Software Systems, Ltd.
Lines: 22

In article <1993Apr20.203344.8417@cs.cornell.edu> karr@cs.cornell.edu (David Karr) writes:
>In article <Clarke.6.735328328@bdrc.bd.com> Clarke@bdrc.bd.com (Richard Clarke) writes:
>>So how do I steer when my hands aren't on the bars? (Open Budweiser in left 
>>hand, Camel cigarette in the right, no feet allowed.) 
>
>>If I lean, and the 
>>bike turns, am I countersteering?
>
>No, the bars would turn only *toward* the direction of turn in
>no-hands steering.

Just in case the original poster was looking for a serious answer,
I'll supply one.

Yes, even when steering no hands you do something quite similar
to countersteering.  Basically to turn left, you to a quick wiggle
of the bike to the right first, causing a counteracting lean to
occur to the left.  It is a lot more difficult to do on a motorcycle
than a bicycle though, because of the extra weight. 

### Extraction Train/Test Data

In [9]:
train_news = fetch_20newsgroups(
    subset = 'train',
    random_state = 2021,
    remove = ('headers', 'footers', 'quotes')
)
len(train_news.data)

11314

In [10]:
test_news = fetch_20newsgroups(
    subset = 'test',
    random_state = 2021,
    remove = ('headers', 'footers', 'quotes')
)
len(test_news.data)

7532

In [11]:
print(train_news.data[1])

]Is it possible to do a "wheelie" on a motorcycle with shaft-drive?

yes.



In [12]:
print(test_news.data[2])

Total Baseball, which also tries to evaluate a player's total offensive
and defensive contributions, gives Barry Bonds a Total Player rating of
9.0 for 1992. Only one other player since Ruth attained that mark: Cal
Ripken for his 1984 season. Rounding out the top five offensive players
in the NL last season: Sheffield, 5.9; Sandberg, 5.8; Van Slyke, 5.3; and
Larkin, 4.7.

The top 5 offensive players in the AL in 1992 were: E. Martinez, 4.8;
Ventura, 4.8; Anderson, 4.5; R. Henderson, 4.4; and Thomas, 4.4

In short, Total Baseball says Bonds enjoyed one of the best seasons
ever in the game's history -- better than any year had by Mantle,
Mays, Williams, etc. (If you disagree, don't flame me; flame the
writers of Total Baseball.) Also, Martinez and Ventura are neck and
neck, so given the shortcomings of any statistical analysis, which one
had the better year can be considered a toss-up. Thus, Total Baseball
supports your choices of Bonds and Ventura as the MVPs of 1992.



--
------------

### Preprocessing Text Data

In [13]:
train_df = pd.DataFrame({'article' : train_news.data})
test_df = pd.DataFrame({'article' : test_news.data})

#### Train Dataset

##### Remove Special Character

In [14]:
train_df['article'] = train_df.article.str.replace('[^A-Za-z]', ' ')

In [15]:
train_df.article[1]

' Is it possible to do a  wheelie  on a motorcycle with shaft drive   yes  '

##### Remove Words with less than Three Characters & Convert to Lowercase

In [16]:
train_df['article'] = train_df.article.apply(lambda x: ' '.join([w.lower() for w in x.split() if len(w) > 3]))

#### Test Dataset

In [17]:
test_df['article'] = test_df.article.str.replace('[^A-Za-z]', ' ')
test_df['article'] = test_df.article.apply(lambda x: ' '.join([w.lower() for w in x.split() if len(w) > 3]))

### Conversion Text Data

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tvect = TfidfVectorizer(stop_words = 'english')
tvect.fit(train_df.article)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [20]:
X_train = tvect.transform(train_df.article)
X_test = tvect.transform(test_df.article)
X_train.shape, X_test.shape

((11314, 64133), (7532, 64133))

In [21]:
y_train = train_news.target
y_test = test_news.target

### Train/Predict/Evaluation

#### Support Vector Machine Classifier

In [22]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
pred = svc.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.6488316516197558